In [1]:
# Import necessary libraries
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the pre-trained tokenizer and model
checkpoint = "mr4/phobert-base-vi-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# Load the Excel file and select the "Draw data" sheet
filename = r"C:\Users\sangnguyendinh\Desktop\CS\Master Data CS FnB.xlsx"
df = pd.read_excel(filename, sheet_name="Draw data")

In [2]:
# Define the column numbers for the Note column and the 3 new result columns
note_col_num = 0  # Assuming Note column is in column A
pos_col_num = len(df.columns)
neg_col_num = len(df.columns) + 1
neu_col_num = len(df.columns) + 2

In [3]:
# Analyze the Note column and write results to new columns
positive = []
negative = []
neutral = []
for note in df["Note"]:
    if pd.isnull(note):
        positive.append("N/A")
        negative.append("N/A")
        neutral.append("N/A")
    else:
        inputs = tokenizer(note, padding=True, truncation=True, return_tensors="pt")
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1).detach().numpy()[0]
        positive.append(round(predictions[1], 3))
        negative.append(round(predictions[0], 3))
        neutral.append(round(predictions[2], 3))

df["Positive"] = positive
df["Negative"] = negative
df["Neutral"] = neutral

In [5]:
# Save the updated dataframe to a new Excel file
df.to_excel(r"C:\Users\sangnguyendinh\Desktop\CS\Phân tích kết quả.xlsx", sheet_name="Draw data", index=False, encoding='utf-8-sig')